In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

# Read from raw data
These data are crawled from <https://en.tutiempo.net>
<br>Youtube on crawling data is as follow: <https://youtu.be/m7Lqda_E3Fg>

In [2]:
# Read from Raw Data Folder
DIR = '../../A_Raw_Data/WeatherData/Weather_WebCrawl'
filelist = os.listdir(DIR)

In [3]:
# read all files and put them in a list
df_list = []
for file in filelist[:]:
    filepath = os.path.join(DIR, file)
    df_list.append(pd.read_csv(filepath)) 

In [4]:
print(f'Total {len(df_list)} files')
print('Example data:')
display(df_list[0].head())

Total 108 files
Example data:


,country,location,station_id,latitude,longitude,Day,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,RA,SN,TS,FG
0,indonesia,Alor / Mali,ws-973200,-8.21,124.56,1-01-2010,29.1,32.2,25,1007.3,84,0,12.7,10,29.4,-,,,o,
1,indonesia,Alor / Mali,ws-973200,-8.21,124.56,2-01-2010,28.7,31.6,24.4,1008.1,80,0,13.2,7.2,18.3,-,,,o,
2,indonesia,Alor / Mali,ws-973200,-8.21,124.56,3-01-2010,27.8,32.4,24.3,1008.2,84,0.51,12.9,11.7,18.3,-,o,,,
3,indonesia,Alor / Mali,ws-973200,-8.21,124.56,4-01-2010,27.2,30.8,23.6,1007.8,87,-,12.6,2.2,13,-,o,,o,
4,indonesia,Alor / Mali,ws-973200,-8.21,124.56,5-01-2010,25.8,27.8,23.8,1008.3,96,67.82,9.7,3.5,9.4,-,o,,o,


# Data Dictionary

In [5]:
data_dict = dict(
    T = 'Average Temperature (°C)',
    TM = 'Maximum temperature (°C)',
    Tm = 'Minimum temperature (°C)',
    SLP = 'Atmospheric pressure at sea level (hPa)',
    H = 'Average relative humidity (%)',
    PP = 'Total rainfall and / or snowmelt (mm)',
    VV = 'Average visibility (Km)',
    V = 'Average wind speed (Km/h)',
    VM = 'Maximum sustained wind speed (Km/h)',
    VG = 'Maximum speed of wind (Km/h)',
    RA = 'Indicate if there was rain or drizzle (In the monthly average, total days it rained)',
    SN = 'Snow indicator (In the monthly average, total days that snowed)',
    TS = 'Indicates whether there storm (In the monthly average, Total days with thunderstorm)',
    FG = 'Indicates whether there was fog (In the monthly average, Total days with fog)'
)
display(pd.DataFrame(data_dict, index= np.arange(1)).T.reset_index().rename(columns={'index': 'name', 0:'description'}))

,name,description
0,T,Average Temperature (°C)
1,TM,Maximum temperature (°C)
2,Tm,Minimum temperature (°C)
3,SLP,Atmospheric pressure at sea level (hPa)
4,H,Average relative humidity (%)
5,PP,Total rainfall and / or snowmelt (mm)
6,VV,Average visibility (Km)
7,V,Average wind speed (Km/h)
8,VM,Maximum sustained wind speed (Km/h)
9,VG,Maximum speed of wind (Km/h)


# Data Cleaning

### 1) Aggregate all data into one single dataframe

In [6]:
df = pd.concat(df_list, axis=0, sort=False).reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385280 entries, 0 to 385279
Data columns (total 21 columns):
index         385280 non-null int64
country       385280 non-null object
location      385280 non-null object
station_id    385280 non-null object
latitude      385141 non-null float64
longitude     385141 non-null float64
Day           385141 non-null object
T             385141 non-null object
TM            385141 non-null object
Tm            385141 non-null object
SLP           385141 non-null object
H             385141 non-null object
PP            385141 non-null object
VV            385141 non-null object
V             385141 non-null object
VM            385141 non-null object
VG            385141 non-null object
RA            385141 non-null object
SN            385141 non-null object
TS            385141 non-null object
FG            385141 non-null object
dtypes: float64(2), int64(1), object(18)
memory usage: 61.7+ MB


### 2) Drop data that have all Null Attributes

In [7]:
index_allnull = df.iloc[:,4:].isna().all(axis=1)
print(f'Total {index_allnull.sum()} rows with all Null attributes')
# Drop them
df = df[~index_allnull]

Total 139 rows with all Null attributes


### 3) set correct type and standardize attributes values

* 3a: Convert column RA, SN, TS, and FG to boolean type with 'o' == True else False

In [8]:
for col in ['RA', 'SN', 'TS', 'FG']:
    df[col] = df[col].apply(lambda x: x=='o')
df.head()

,index,country,location,station_id,latitude,longitude,Day,T,TM,Tm,...,H,PP,VV,V,VM,VG,RA,SN,TS,FG
0,0,indonesia,Alor / Mali,ws-973200,-8.21,124.56,1-01-2010,29.1,32.2,25,...,84,0,12.7,10,29.4,-,False,False,True,False
1,1,indonesia,Alor / Mali,ws-973200,-8.21,124.56,2-01-2010,28.7,31.6,24.4,...,80,0,13.2,7.2,18.3,-,False,False,True,False
2,2,indonesia,Alor / Mali,ws-973200,-8.21,124.56,3-01-2010,27.8,32.4,24.3,...,84,0.51,12.9,11.7,18.3,-,True,False,False,False
3,3,indonesia,Alor / Mali,ws-973200,-8.21,124.56,4-01-2010,27.2,30.8,23.6,...,87,-,12.6,2.2,13,-,True,False,True,False
4,4,indonesia,Alor / Mali,ws-973200,-8.21,124.56,5-01-2010,25.8,27.8,23.8,...,96,67.82,9.7,3.5,9.4,-,True,False,True,False


* 3b: Convert all attribute value '-' to np.nan

In [9]:
df = df.applymap(lambda x: np.nan if x=='-' else x)
df.head()

,index,country,location,station_id,latitude,longitude,Day,T,TM,Tm,...,H,PP,VV,V,VM,VG,RA,SN,TS,FG
0,0,indonesia,Alor / Mali,ws-973200,-8.21,124.56,1-01-2010,29.1,32.2,25,...,84,0,12.7,10,29.4,NaN,False,False,True,False
1,1,indonesia,Alor / Mali,ws-973200,-8.21,124.56,2-01-2010,28.7,31.6,24.4,...,80,0,13.2,7.2,18.3,NaN,False,False,True,False
2,2,indonesia,Alor / Mali,ws-973200,-8.21,124.56,3-01-2010,27.8,32.4,24.3,...,84,0.51,12.9,11.7,18.3,NaN,True,False,False,False
3,3,indonesia,Alor / Mali,ws-973200,-8.21,124.56,4-01-2010,27.2,30.8,23.6,...,87,NaN,12.6,2.2,13,NaN,True,False,True,False
4,4,indonesia,Alor / Mali,ws-973200,-8.21,124.56,5-01-2010,25.8,27.8,23.8,...,96,67.82,9.7,3.5,9.4,NaN,True,False,True,False


* 3c: Convert columns T, TM, Tm, SLP, H, PP, VV, V, VM and VG to float type

In [10]:
for col in ['T', 'TM', 'Tm', 'SLP', 'H', 'PP', 'VV', 'V', 'VM', 'VG']:
    df[col] = df[col].astype(float)

* 3d: Convert column Day to datetime object

In [11]:
df['Day'] = pd.to_datetime(df['Day'], format='%d-%m-%Y')

* 3e: Convert columns country, location and station_id to categorical data

In [12]:
for col in ['country', 'location', 'station_id']:
    df[col] = df[col].astype('category')

### 4) Get Summary Statistic of Data's numerical attributes, and drop attribute with excessing missing data

In [13]:
df.describe()

,index,latitude,longitude,T,TM,Tm,SLP,H,PP,VV,V,VM,VG
count,385141.000000,385141.00000,385141.000000,377386.000000,377338.000000,377266.000000,316055.000000,377222.000000,335193.000000,377156.000000,377351.000000,369781.000000,8272.000000
mean,1812.045672,-1.16437,114.604139,27.268288,31.502685,23.526370,1010.085454,81.069418,4.966171,8.937719,6.630251,15.371745,39.453808
std,1053.408189,4.69231,11.872902,1.772392,2.097219,2.022226,2.072144,6.765546,13.581008,2.297175,3.888401,6.832759,8.899469
min,0.000000,-10.73000,95.310000,3.900000,6.000000,-1.000000,996.500000,28.000000,0.000000,0.200000,0.000000,1.900000,20.600000
25%,899.000000,-4.10000,103.660000,26.600000,30.500000,23.000000,1008.800000,77.000000,0.000000,7.400000,4.100000,11.100000,33.500000
50%,1804.000000,-1.00000,113.980000,27.500000,31.800000,23.900000,1010.000000,82.000000,0.000000,9.000000,5.900000,14.800000,38.900000
75%,2723.000000,3.10000,122.960000,28.300000,33.000000,24.600000,1011.200000,86.000000,3.050000,10.100000,8.300000,18.300000,44.600000
max,3651.000000,6.91000,140.710000,32.900000,43.800000,30.000000,1028.400000,100.000000,406.910000,27.700000,56.100000,111.100000,92.400000


In [14]:
percent_missing = (df.isna().sum(axis=0)*100/len(df)).rename('missing').to_frame().sort_values('missing',  ascending=False).T
display(percent_missing)

,VG,SLP,PP,VM,VV,H,Tm,TM,V,T,...,SN,RA,index,country,Day,longitude,latitude,station_id,location,FG
missing,97.852215,17.937846,12.968757,3.98815,2.073267,2.05613,2.044706,2.026011,2.022636,2.013548,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* We can see column VG has more than 97% of missing value. We will drop this column
* SLP with ~18 percent missing data seems high too. We will drop this column as well


In [15]:
df = df.drop(['VG', 'SLP'], axis=1)

* Next we group data by location and see if any missing value is more than 30% of the data. There are None, so we will work with these mising value by KNN imputation

### 5) Median imputation

In [17]:
na_index = df.isna().any(axis=1)
unique_stations = df['station_id'].unique()
df.set_index('Day', inplace=True)

df_list = []
knnimputer = KNNImputer(n_neighbors=11)
for station in unique_stations:
    data = df[df['station_id']==station]
    imputed_df = pd.DataFrame(knnimputer.fit_transform(data.iloc[:,4:]), columns= data.iloc[:,4:].columns)
    df_list.append(pd.concat([data.iloc[:,:4].reset_index(), imputed_df], axis=1))


In [ ]:
data